In [2]:
import numpy as np
import cvxpy as cp
from sdp_data_matrix_scalable import smooth_convex_gd, generate_sample

In [3]:
d = 5
K = 5
mu = 0.0
L = 1.0
gamma = 1/L

A, b, C = smooth_convex_gd(d, K, mu, L, gamma)
b = np.array(b)

#### One sample $(N = 1)$ and linear loss $\ell$ $(J = 1)$

In [4]:
sample_n = 1
sample_xks, sample_gks, sample_fks = generate_sample(d, K, mu, L, gamma, seed=0)
sample_P = sample_xks[0]
for g in sample_gks[:-1] :
    sample_P = np.concatenate([sample_P, g], axis=1)
sample_G = sample_P.T @ sample_P
sample_F = np.concatenate(sample_fks, axis=1)[0]
print(sample_G)
print(sample_F)

[[1.00000000e+00 7.61312803e-01 1.11074391e-01 2.28737071e-02
  4.82355472e-03 1.09104429e-03]
 [7.61312803e-01 6.50238412e-01 8.82006841e-02 1.80501524e-02
  3.73251044e-03 7.95997382e-04]
 [1.11074391e-01 8.82006841e-02 1.80501524e-02 3.73251044e-03
  7.95997382e-04 1.85545052e-04]
 [2.28737071e-02 1.80501524e-02 3.73251044e-03 7.95997382e-04
  1.85545052e-04 5.34789090e-05]
 [4.82355472e-03 3.73251044e-03 7.95997382e-04 1.85545052e-04
  5.34789090e-05 2.15364504e-05]
 [1.09104429e-03 7.95997382e-04 1.85545052e-04 5.34789090e-05
  2.15364504e-05 1.16522745e-05]]
[3.80656402e-01 1.14368536e-02 5.45522143e-04 5.47509267e-05
 1.72432470e-05 0.00000000e+00]


In [5]:
G_shape = C.G.shape
F_shape = C.F.shape
m = len(b)
G_shape, F_shape, m

((6, 6), (6,), 41)

In [6]:
sample_G.shape, sample_F.shape

((6, 6), (6,))

In [7]:
lmbd = cp.Variable()
s = cp.Variable()
G = cp.Variable(G_shape, symmetric=True)
F = cp.Variable(F_shape)
y = cp.Variable(m)
Aopt = -C.G
bopt = -C.F
c = -b

constraint = [- c.T@y - cp.trace(G@sample_G) - F.T@sample_F <= s]
constraint += [cp.SOC(lmbd, cp.hstack([cp.vec(G), F]))]
LstarG = np.zeros(G_shape)
LstarF = np.zeros(F_shape)
for i in range(m) :
    LstarG = LstarG + y[i]*A[i].G
    LstarF = LstarF + y[i]*A[i].F
constraint += [LstarG - G - Aopt >> 0]
constraint += [LstarF - F - bopt == 0]
constraint += [y >= 0]

##### When `eps=0.0`, it is the same as the sample convergence bound. When `eps` super large, it is the same as the PEP bound.

In [15]:
# eps = 0.0 (sample) <-------> eps = 1e10 (PEP)
eps = 1
prob = cp.Problem(cp.Minimize(lmbd*eps + 1/sample_n * cp.sum(s)), constraint)
prob.solve(solver=cp.CLARABEL)
print('optimal value (DRO-PEP) =', prob.value)
print('cvxpy status =', prob.status)
# print(lmbd.value)
# print(s.value)

optimal value (DRO-PEP) = 0.04910434197587718
cvxpy status = optimal


In [9]:
sample_F

array([3.80656402e-01, 1.14368536e-02, 5.45522143e-04, 5.47509267e-05,
       1.72432470e-05, 0.00000000e+00])

#### compare with PEP result

In [10]:
G_PEP = cp.Variable(C.G.shape, symmetric=True)
F_PEP = cp.Variable(C.F.shape)

constraint_PEP = [G_PEP >> 0]
constraint_PEP += [F_PEP >= 0.0]
for i in range(m) :
    constraint_PEP += [cp.trace(G_PEP@A[i].G) + F_PEP.T@A[i].F <= b[i]]
prob = cp.Problem(cp.Maximize(- cp.trace(G_PEP@C.G) - F_PEP.T@C.F), constraint_PEP)
prob.solve(solver=cp.CLARABEL)
print('optimal value (PEP) =', prob.value)
print('cvxpy status =', prob.status)

optimal value (PEP) = 0.05555555532321356
cvxpy status = optimal
